In [ ]:
import numpy as np
import pandas as pd

import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.lang.en.stop_words import STOP_WORDS
STOP_WORDS.remove('call')

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

import random
from random import shuffle
random.seed(1)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wEBzLZxMm3fYX1JQqUgm28-5_Yos67NDn4_nTfWPtfl1JHSDGtQsYE
Mounted at /content/drive


In [ ]:
PATH = '/content/'
FILE = f'{PATH}stopwords.xlsx'

X0 = 'Final Data'
X1 = 'Short description'
X2 = 'Description'
X3 = 'Caller'
Y = 'Assignment group'

X4 = 'Word Count'
X5 = 'Short Word Count'

In [ ]:
def merge_data(short_desc, long_desc):
    if str(long_desc).find(str(short_desc)) != -1:
        return str(long_desc).lower()
    else:
        return (str(short_desc) + " " + str(long_desc)).lower()

In [ ]:
data = None
data = pd.read_excel(FILE, engine='openpyxl')
data[X0] = data.apply(lambda x: merge_data(x[X1], x[X2]), axis=1)
data[X4] = data[X0].str.split(" ").apply(lambda x: len(x))
data[X5] = data[X1].astype(str).str.split(" ").apply(lambda x: len(x))

In [ ]:
data.head()

,Unnamed: 0,Short description,Description,Caller,Assignment group,Final Data,Word Count,Short Word Count
0,0,login issue,verified user details employee manager checked...,spxjnwir pjlcoqds,GRP_0,verified user details employee manager checked...,20,2
1,1,outlook,received hello team meetings skype meetings et...,hmjdrvpb komuaywn,GRP_0,received hello team meetings skype meetings et...,14,1
2,2,nt log vpn,received hi log vpn best,eylqgodm ybqkwiam,GRP_0,nt log vpn received hi log vpn best,8,3
3,3,unable access hr tool page,unable access hr tool page,xbkucsvz gcpydteq,GRP_0,unable access hr tool page,5,5
4,4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,skype error,2,2


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8492 entries, 0 to 8491
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         8492 non-null   int64 
 1   Short description  8478 non-null   object
 2   Description        8425 non-null   object
 3   Caller             8492 non-null   object
 4   Assignment group   8492 non-null   object
 5   Final Data         8492 non-null   object
 6   Word Count         8492 non-null   int64 
 7   Short Word Count   8492 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 530.9+ KB


In [ ]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,8492.0,4245.500000,2451.573576,0.0,2122.75,4245.5,6368.25,8491.0
Word Count,8492.0,18.156500,41.855103,1.0,5.00,9.0,19.00,1062.0
Short Word Count,8492.0,4.962789,2.570276,1.0,3.00,4.0,6.00,19.0


In [ ]:
data.shape

(8492, 8)

The simple data augmentation techniques are the following:

1. SR: synonym replacement
2. RD: random deletion
3. RS: random swap
4. RI: random insertion

From Maël Fabien

In [ ]:
def get_synonyms(word):
  synonyms = set()
    
  for syn in wordnet.synsets(word): 
    for l in syn.lemmas(): 
      synonym = l.name().replace("_", " ").replace("-", " ").lower()
      synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
      synonyms.add(synonym) 
    
  if word in synonyms:
    synonyms.remove(word)
    
  return list(synonyms)

#######################################################################################

def synonym_replacement(words, n):
  new_words = words.copy()
  random_word_list = list(set([word for word in words if word not in STOP_WORDS]))
  random.shuffle(random_word_list)
  num_replaced = 0
    
  for random_word in random_word_list:
    synonyms = get_synonyms(random_word)
        
    if len(synonyms) >= 1:
      synonym = random.choice(list(synonyms))
      new_words = [synonym if word == random_word else word for word in new_words]
      num_replaced += 1
        
    if num_replaced >= n: #only replace up to n words
      break

  sentence = ' '.join(new_words)
  
  return sentence

#######################################################################################
def random_deletion(words, p):
  #obviously, if there's only one word, don't delete it
  if len(words) == 1:
    return words

  #randomly delete words with probability p
  new_words = []
  for word in words:
    r = random.uniform(0, 1)
    if r > p:
      new_words.append(word)

  #if you end up deleting all words, just return a random word
  if len(new_words) == 0:
    rand_int = random.randint(0, len(words)-1)
    return [words[rand_int]]

  sentence = ' '.join(new_words)
  return sentence

#######################################################################################

def swap_word(new_words):
  random_idx_1 = random.randint(0, len(new_words)-1)
  random_idx_2 = random_idx_1
  counter = 0
    
  while random_idx_2 == random_idx_1:
    random_idx_2 = random.randint(0, len(new_words)-1)
    counter += 1
        
    if counter > 3:
      return new_words
    
  new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
  return new_words

#######################################################################################

def random_swap(words, n):
  new_words = words.copy()
    
  for _ in range(n):
    new_words = swap_word(new_words)
        
  sentence = ' '.join(new_words)
  return sentence

#######################################################################################

def random_insertion(words, n):
  new_words = words.copy()
  
  for _ in range(n):
    add_word(new_words)
        
  sentence = ' '.join(new_words)
  return sentence

#######################################################################################

def add_word(new_words):  
  synonyms = []
  counter = 0
    
  while len(synonyms) < 1:
    random_word = new_words[random.randint(0, len(new_words)-1)]
    synonyms = get_synonyms(random_word)
    counter += 1
    if counter >= 10:
      return
        
  random_synonym = synonyms[0]
  random_idx = random.randint(0, len(new_words)-1)
  new_words.insert(random_idx, random_synonym)

In [ ]:
def eda_data_augment(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=15):
  words = sentence.split(' ')
  words = [word for word in words if word is not '']
  num_words = len(words)

  augmented_sentences = []
  num_new_per_technique = int(num_aug / 4) + 1
  n_sr = max(1, int(alpha_sr * num_words))
  n_ri = max(1, int(alpha_ri * num_words))
  n_rs = max(1, int(alpha_rs * num_words))

  #sr
  for _ in range(num_new_per_technique):
    a_words = synonym_replacement(words, n_sr)
    augmented_sentences.append(''.join(a_words))
  
  #ri
  for _ in range(num_new_per_technique):
    a_words = random_insertion(words, n_ri)
    augmented_sentences.append(''.join(a_words))

  #rs
  for _ in range(num_new_per_technique):
    a_words = random_swap(words, n_rs)
    augmented_sentences.append(''.join(a_words))

  #rd
  for _ in range(num_new_per_technique):
    a_words = random_deletion(words, p_rd)
    augmented_sentences.append(''.join(a_words))

  augmented_sentences = [sentence for sentence in augmented_sentences]
  shuffle(augmented_sentences)

  #trim so that we have the desired number of augmented sentences
  if num_aug >= 1:
    augmented_sentences = augmented_sentences[:num_aug]
  else:
    keep_prob = num_aug / len(augmented_sentences)
    augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < 1]

  #append the original sentence
  # augmented_sentences.append(sentence)

  return augmented_sentences

In [ ]:
need_augmentation = data.groupby(Y).filter(lambda x: len(x) < 16)

In [ ]:
data.shape

(8492, 8)

In [ ]:
augmented_series = []
for index, row in need_augmentation.iterrows():
  augmented_sentences = eda_data_augment(row[X0])
  for sentence in augmented_sentences:
    augmented_series.append(pd.DataFrame.from_records([{X0: sentence, X1: row[X1], X2: row[X2], X3: row[X3], X4: len(sentence.split(' ')), X5: row[X5], Y: row[Y]}]))
data = data.append(augmented_series)
data.reset_index(inplace=True)

In [ ]:
data.shape

(10787, 9)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10787 entries, 0 to 10786
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Short description  10773 non-null  object
 1   Description        10720 non-null  object
 2   Caller             10787 non-null  object
 3   Assignment group   10787 non-null  object
 4   Final Data         10787 non-null  object
 5   Word Count         10787 non-null  int64 
 6   Short Word Count   10787 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 590.0+ KB


In [ ]:
data.drop(columns=['index', 'Unnamed: 0'], axis=1, inplace=True)

In [ ]:
data.head()

,Short description,Description,Caller,Assignment group,Final Data,Word Count,Short Word Count
0,login issue,verified user details employee manager checked...,spxjnwir pjlcoqds,GRP_0,verified user details employee manager checked...,20,2
1,outlook,received hello team meetings skype meetings et...,hmjdrvpb komuaywn,GRP_0,received hello team meetings skype meetings et...,14,1
2,nt log vpn,received hi log vpn best,eylqgodm ybqkwiam,GRP_0,nt log vpn received hi log vpn best,8,3
3,unable access hr tool page,unable access hr tool page,xbkucsvz gcpydteq,GRP_0,unable access hr tool page,5,5
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,skype error,2,2


In [ ]:
data.to_excel('data_augmented_post_stopword.xlsx')